In [1]:
import yaml
from google.cloud import bigquery


In [ ]:
config = yaml.load(open("credentials.yml"), yaml.Loader)
client = bigquery.Client.from_service_account_json(config["credentials_path"])


In [9]:
dataset_ref = bigquery.DatasetReference("bigquery-public-data", "hacker_news")
table_ref = bigquery.TableReference(dataset_ref, "full")

table = client.get_table(table_ref)
client.list_rows(table, max_results=5).to_dataframe()


,title,url,text,dead,by,score,time,timestamp,type,id,parent,descendants,ranking,deleted
0,None,None,"I would rather just have wired earbuds, period...",<NA>,zeveb,<NA>,1591717736,2020-06-09 15:48:56+00:00,comment,23467666,23456782,<NA>,<NA>,<NA>
1,None,None,DNS?,<NA>,nly,<NA>,1572810465,2019-11-03 19:47:45+00:00,comment,21436112,21435130,<NA>,<NA>,<NA>
2,None,None,These benchmarks seem pretty good. Filterable...,<NA>,mrkeen,<NA>,1591717727,2020-06-09 15:48:47+00:00,comment,23467665,23467426,<NA>,<NA>,<NA>
3,None,None,Oh really?<p>* Excel alone uses 86.1MB of priv...,<NA>,oceanswave,<NA>,1462987532,2016-05-11 17:25:32+00:00,comment,11677248,11676886,<NA>,<NA>,<NA>
4,None,None,These systems are useless. Of the many flaws:...,<NA>,nyxxie,<NA>,1572810473,2019-11-03 19:47:53+00:00,comment,21436113,21435025,<NA>,<NA>,<NA>


In [28]:
query = """
    SELECT parent, COUNT(id)
    FROM `bigquery-public-data.hacker_news.full`
    GROUP BY parent
    HAVING COUNT(id) > 10
"""

safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**9)  # 1 GB
query_job = client.query(query, job_config=safe_config)
query_results = query_job.to_dataframe()
query_results.head()


,parent,f0_
0,22545878,51
1,14620608,70
2,29067491,315
3,14901313,775
4,33573246,79


In [29]:
query = """
    SELECT parent, COUNT(1) AS NumPosts
    FROM `bigquery-public-data.hacker_news.full`
    GROUP BY parent
    HAVING COUNT(1) > 10
    ORDER BY COUNT(1) DESC
"""

safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**9)  # 1 GB
query_job = client.query(query, job_config=safe_config)
query_results = query_job.to_dataframe()
query_results.head()


,parent,NumPosts
0,<NA>,4723217
1,363,1318
2,23170881,1105
3,30934529,1051
4,27355392,1041


In [30]:
# Write a query that returns all authors with more than 10,000 posts as well
# as their post counts. Call the column with post counts `NumPosts`.
query = """
    SELECT `by`, COUNT(1) AS NumPosts
    FROM `bigquery-public-data.hacker_news.full`
    GROUP BY `by`
    HAVING COUNT(1) > 10000
"""

safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**9)
query_job = client.query(query, job_config=safe_config)
query_results = query_job.to_dataframe()
query_results.head()


,by,NumPosts
0,sp332,18565
1,icebraining,22889
2,detaro,21631
3,derefr,13381
4,pavel_lishin,11034


In [31]:

# How many comments have been deleted? (If a comment was deleted, the
# `deleted` column in the comments table will have the value `True`.)

query = """
    SELECT deleted, COUNT(1) AS num_deleted_posts
    FROM `bigquery-public-data.hacker_news.full`
    GROUP BY deleted
    HAVING deleted = True
"""

query = """
    SELECT COUNT(1) AS num_deleted_posts
    FROM `bigquery-public-data.hacker_news.full`
    WHERE deleted = True
"""

safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**9)
query_job = client.query(query, job_config=safe_config)
query_results = query_job.to_dataframe()
query_results.head()


,num_deleted_posts
0,968172
